<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-project/blob/main/BLM6114_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BLM6114 Hesaplamalı Anlambilim Dersi Dönem Sonu Projesi

## Ortam Kurulumu

In [3]:
!pip install -q transformers datasets scikit-learn umap-learn matplotlib huggingface-hub sentence-transformers
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00


## Huggingface login

In [4]:
from google.colab import userdata
token = userdata.get('HF_TOKEN')

from huggingface_hub import login
login(token=token)

## Veri seti işlemleri

In [5]:
from huggingface_hub import hf_hub_download
import pandas as pd

df = pd.read_csv("hf://datasets/ytu-ce-cosmos/gsm8k_tr/gsm8k_tr.csv")
print(df.columns)
df.head()


Index(['question', 'answer'], dtype='object')


,question,answer
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...


## Soru türü ve cevap yöntemi tespit etme

In [17]:
import os
import pandas as pd
import openai
import json
from google.colab import userdata

token = userdata.get('OPEN_AI_API_KEY')

# Veri kümesini yükleme
df = pd.read_csv("hf://datasets/ytu-ce-cosmos/gsm8k_tr/gsm8k_tr.csv")

# API anahtarını ayarla
client = openai.OpenAI(api_key=token)

def classify_question_and_method(question: str, answer: str) -> str:
    """
    İki cümle döndürür:
    1) Sorunun türü
    2) Çözüm yöntemi
    """

    prompt = (
    f"Soru: {question}\n"
    f"Cevap: {answer}\n\n"
    "Lütfen **yalnızca JSON** formatında cevapla ve şu iki alanı içersin:\n"
    "{\n"
    '  "question_type": "<kısa ve net bir cümle olarak sorunun türü (“toplama”, “oran-orantı” vb.)>",\n'
    '  "solution_method": "<kısa ve net bir cümle olarak çözüm yöntemi (“denklem kurma”, “birim oran” vb.)>"\n'
    "}\n"
    "Cevapta başka metin veya açıklama OLMAMALI."
    )

    resp = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user','content':prompt}],
        temperature=0
    )
    result = resp.choices[0].message.content.strip()
    json_string = "\n".join(result.strip().splitlines()[1:-1])
    data = json.loads(json_string)
    return data['question_type'], data['solution_method']

# Örnek için ilk 5 kayıt
sample = df.head(5).copy()
sample["question_type"], sample["solution_method"] = zip(*sample.apply(
    lambda row: classify_question_and_method(row['question'], row['answer']), axis=1
))

sample.head()

# -- Tüm veri kümesi için --
# df['classification'] = df.apply(
#     lambda row: classify_question_and_method(row['question'], row['answer']), axis=1
# )
# df.to_csv('gsm8k_tr_classified.csv', index=False)


,question,answer,question_type,solution_method
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,oran-orantı,yüzde hesaplama
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,yaş problemi,doğrudan çıkarma
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,oran-orantı,birim oran
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,çarpma,doğrudan hesaplama
4,Mark'ın iki evcil hayvanı var: Saniyede 10 adı...,Kaplumbağanın yarışı berabere bitirmesi için 1...,oran-orantı,birim oran
